In [ ]:
import tensorflow as tf

(train_images, train_labels), (_, _) = tf.keras.datasets.mnist.load_data()

2024-12-06 14:54:08.745871: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-06 14:54:11.846349: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-06 14:54:13.403754: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733496855.834809    2877 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733496856.612041    2877 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-06 14:54:21.895538: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

In [2]:
#Defining batch size for later
BATCH_SIZE = 256

train_images = train_images.reshape(train_images.shape[0], 28, 28, 1).astype('float32')
train_images = (train_images - 127.5) / 127.5  # Normalize the images to [-1, 1]

In [3]:
print(train_images.shape)

(60000, 28, 28, 1)


## Step 2

GANs are made of two models: A Generator and a Discriminator. The generator makes the images, while the discriminator gives feedback to the generator on whether the images are convincingly fake or not

In [4]:
from tensorflow.keras.layers import *

def make_generator():
    model = tf.keras.Sequential()

    #Dense layer to upscale noise
    model.add(Dense(7*7*256, use_bias=False, input_shape=(100,)))
    model.add(BatchNormalization())
    model.add(LeakyReLU())
    model.add(Reshape((7, 7, 256)))
    
    #First Conv2D layer
    model.add(Conv2DTranspose(128, (5, 5), strides=(1, 1), padding='same', use_bias=False))
    model.add(BatchNormalization())
    model.add(LeakyReLU())

    #Second Conv2D layer
    model.add(Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    model.add(BatchNormalization())
    model.add(LeakyReLU())

    #Final Conv2D layer
    model.add(Conv2DTranspose(1, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh'))

    return model


In [5]:
generator = make_generator()
generator.summary()

/usr/local/python/3.12.1/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 12544)          │     1,254,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 12544)          │        50,176 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu (LeakyReLU)         │ (None, 12544)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape (Reshape)               │ (None, 7, 7, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose                │ (None, 7, 7, 128)      │       819,200 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 7, 7, 128)      │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_1 (LeakyReLU)       │ (None, 7, 7, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_1              │ (None, 14, 14, 64)     │       204,800 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 14, 14, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_2 (LeakyReLU)       │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_2              │ (None, 28, 28, 1)      │         1,600 │
│ (Conv2DTranspose)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,330,944 (8.89 MB)

 Trainable params: 2,305,472 (8.79 MB)

 Non-trainable params: 25,472 (99.50 KB)

In [6]:
def build_discriminator(image_size=28, num_channels=1):
    model = tf.keras.Sequential()

    # Input: (image_size, image_size, num_channels)
    model.add(Conv2D(64, kernel_size=(5,5), strides=2,
                     input_shape=(image_size, image_size, num_channels)))
    model.add(LeakyReLU())
    model.add(Dropout(0.3))

    # 2nd Convolutional Layer
    model.add(Conv2D(128, kernel_size=(5,5), strides=2, padding="same"))
    model.add(LeakyReLU())
    model.add(Dropout(0.3))

    # Output layer
    model.add(Flatten())
    model.add(Dense(1, activation="sigmoid"))  # Output a single probability

    return model

In [7]:
discriminator = build_discriminator()
discriminator.summary()

/usr/local/python/3.12.1/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 12, 12, 64)     │         1,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_3 (LeakyReLU)       │ (None, 12, 12, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 12, 12, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 6, 6, 128)      │       204,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_4 (LeakyReLU)       │ (None, 6, 6, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 6, 6, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 4608)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │         4,609 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 211,201 (825.00 KB)

 Trainable params: 211,201 (825.00 KB)

 Non-trainable params: 0 (0.00 B)

In [8]:
import numpy as np
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt


class GAN:
    def __init__(self, generator, discriminator, batch_size=BATCH_SIZE, latent_dims=100):
        self.generator = generator
        self.discriminator = discriminator
        self.BATCH_SIZE = batch_size       
        self.LATENT_DIMS =  latent_dims

        self.discriminator.trainable = False
        self.gan = tf.keras.models.Sequential([self.generator, self.discriminator])

        discriminator_optimizer = Adam(learning_rate=0.0002, beta_1=0.5)
        generator_optimizer = Adam(learning_rate=0.0002, beta_1=0.5)
        
        self.gan.compile(
            loss='binary_crossentropy',
            optimizer=generator_optimizer,
        )

        self.discriminator.trainable = True        
        self.discriminator.compile(
            loss='binary_crossentropy',
            optimizer=discriminator_optimizer,
            metrics=['accuracy']
        )
        

    def get_images(self, num_images):
        noise = np.random.normal(0, 1, (num_images, self.LATENT_DIMS))
        images = self.generator.predict(noise, verbose=0)
        return images

    def train_discriminator(self, data):
        self.discriminator.trainable = True
        idx = np.random.randint(0, data.shape[0], self.BATCH_SIZE)
        real_images = data[idx]

        # Generate fake images
        fake_images = self.get_images(self.BATCH_SIZE)
        #Labels
        real_labels = np.ones((self.BATCH_SIZE, 1))
        fake_labels = np.zeros((self.BATCH_SIZE, 1))
        
        # Train the discriminator
        d_loss_real = self.discriminator.train_on_batch(real_images, real_labels)
        d_loss_fake = self.discriminator.train_on_batch(fake_images, fake_labels)
        d_loss = np.add(d_loss_real, d_loss_fake)
        
        self.discriminator.trainable = False
        return d_loss
    

    def train_generator(self):
        self.generator.trainable = True
        noise = np.random.normal(0, 1, (self.BATCH_SIZE, self.LATENT_DIMS))
        labels = np.ones((self.BATCH_SIZE, 1)) #All ones so that the fake (0) reflects as loss
        
        g_loss = self.gan.train_on_batch(noise, labels)

        self.generator.trainable = False
        return g_loss
    

    def show_generated_image(self, noise):
        fake_image = self.generator.predict(noise, verbose=0)[0]
        fake_image = np.clip((fake_image*127.5)+127.5, 0, 255).astype(np.uint8)  # Denormalize
        plt.imshow(fake_image)
        plt.axis('off')
        plt.show()
        

    def train(self, data, epochs):
        FIXED_LOSS = np.random.normal(0, 1, (1, self.LATENT_DIMS))
        for i in range(epochs):
            disc_loss = self.train_discriminator(data)
            gan_loss = self.train_generator()
            if i%500 == 0:
                print(f"Epoch {i} completed\n\n")

            """if i%100 == 0: #Shows results every 100 epochs
                print(f"Discriminator Loss: {disc_loss}")
                print(f"Generator Loss: {gan_loss}")
                self.show_generated_image(FIXED_LOSS)"""

In [9]:
gan = GAN(generator, discriminator)

In [10]:
gan.train(train_images, 10000)

Epoch 0 completed


Epoch 500 completed


Epoch 1000 completed


Epoch 1500 completed


Epoch 2000 completed


Epoch 2500 completed


Epoch 3000 completed


Epoch 3500 completed


Epoch 4000 completed


Epoch 4500 completed


Epoch 5000 completed


Epoch 5500 completed


Epoch 6000 completed


Epoch 6500 completed


Epoch 7000 completed


Epoch 7500 completed


Epoch 8000 completed


Epoch 8500 completed


Epoch 9000 completed


Epoch 9500 completed




In [11]:
generator.save("model/mnist-generator.keras")